In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from google.colab import drive, files
import io

# 5. Stop words, lematizing, stemming

## Cargar datos y dividir dataset

In [2]:
from tensorflow.random import set_seed
set_seed(234730)

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_without_label.csv')

In [5]:
train['id'] = train['id'].astype(str)
train['tweet'] = train['tweet'].astype(str)
train['label'] = train['label'].astype(str)

In [6]:
train.drop(['id'], axis=1)
train.set_index("id", inplace = True)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(
    train.drop('label', axis=1),
    train['label'],
    test_size=(1.0/3), random_state=42)

## Stop words y lemmatize

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')
Lemmatizer = WordNetLemmatizer()
#analyzer = CountVectorizer().build_analyzer()

from zipfile import ZipFile
file_loc = '/root/nltk_data/corpora/wordnet.zip'
with ZipFile(file_loc, 'r') as z:
  z.extractall('/root/nltk_data/corpora/')

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#Lemmatizer
def lemmatized_words(doc):
  return (Lemmatizer.lemmatize(w.lower()) for w in doc.split())

#CountVectorizer con stopwords de Natural Language Toolkit
nltk.download('stopwords')

nltk.download('omw-1.4')

## CountVectorizer con los nuevos parametros

In [11]:
vectorizer_stop_words_stemmed = CountVectorizer(stop_words = nltk.corpus.stopwords.words('english'), analyzer=lemmatized_words)
bag_of_words = vectorizer_stop_words_stemmed.fit(X_train["tweet"])

print(bag_of_words.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:538: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(


['!', '!!', '!!!', '!☺', '"', '"\'this', '"..in', '"17', '"4', '"5g', '"99%"', '"[t]he', '"a', '"activates"', '"agree', '"akira???', '"allahu', '"almost', '"although', '"antibodies"', '"antidotes."', '"anybody', '"anyone', '"appalling', '"are', '"around', '"arsenicum', '"as', '"because', '"being', '"bill', '"biological', '"bioweapon', '"bought', '"bound', '"burning', '"but', '"cases', '"caused"', '"celebrate"', '"center', '"centre', '"certificate', '"challenge"', '"chaotic".', '"china', '"claire', '"class', '"clean"', '"clearly', '"clo2"', '"clorox', '"colloidal', '"come', '"considerable', '"contagion', '"containment', '"continue', '"corona', '"coronavirus', '"coronavirus"', '"coronavirus."', '"could', '"council', '"cov...,"', '"covid', '"covid-(1).”', '"covid-19', '"covid-19"', '"cumulative', '"cure"', '"cures', '"current"', '"currently', '"dark', '"deadly', '"decision', '"destroy"', '"direct', '"disobeyed', '"doctors', '"doubling"', '"down', '"dramatically', '"drink', '"effective', '

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Punctuación
Hay muchas palabras que estan repetidas como "august", "august,", "august.", "august:", etc. Para resolver esto vamos a sacar la punctuación antes de hacer el lemmatizer. Por ahora vamos a dejar algunos símbolos como los "#", quizas los hastags pueden tener información importante.

Como en el órden de ejecución del CountVectorizer primero se hace el lemmatizing,  no podemos ponerlo en las stop words. Por lo tanto lo vamos a hacer en el mismo método de lemmatizing.

In [ ]:
#All_punct = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
All_punct = '''!()-[]{};:'"\,<>./?%^&*_~'''
#CountVectorizer con stopwords de Natural Language Toolkit
nltk.download('stopwords')

def is_Not_Link(word):
  return word[0:4] != "http"

def remove_Punctuation(doc_split):
  for i in range(len(doc_split)):
    word = doc_split[i];
    if is_Not_Link(word):
      for elements in word:
        if elements in All_punct:
          doc_split[i] = word.replace(elements, "")
  return doc_split

def remove_Stopwords(doc_split):
  doc_with_Stopwords = doc_split.copy();
  for i in range(len(doc_split)):
    word = doc_split[i];
    if word in nltk.corpus.stopwords.words('english'):
      doc_with_Stopwords.remove(word);

  return doc_with_Stopwords

def new_analyzer(doc):
  doc_split = doc.split();
  doc_split = remove_Punctuation(doc_split);
  doc_split = remove_Stopwords(doc_split);
  return (Lemmatizer.lemmatize(w.lower()) for w in doc_split);


#Lemmatizer
def lemmatized_words_punctuation(doc):
  doc_split = doc.split();
  for i in range(len(doc_split)):
    word = doc_split[i];
    if is_Not_Link(word):
      for elements in word:
        if elements in All_punct:
          doc_split[i] = word.replace(elements, "")
  return (Lemmatizer.lemmatize(w.lower()) for w in doc_split)




nltk.download('omw-1.4')

In [76]:
print(nltk.corpus.stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#  if doc[0:4] == "http":
#    return doc
#  else:
#    for elements in doc:
#      if elements in All_punct:
#        doc = doc.replace(elements, "")
#    return (Lemmatizer.lemmatize(w.lower()) for w in doc.split())

In [83]:
vectorizer_stop_words_stemmed = CountVectorizer(stop_words = nltk.corpus.stopwords.words('english'), analyzer=new_analyzer)
bag_of_words = vectorizer_stop_words_stemmed.fit(X_train["tweet"])

print(bag_of_words.get_feature_names())

['', '!', '"[the', '"akira', '"chaotic"', '"covid-(1)”', '"covid19', '"exterminated"', '"hoax"', '"huge', '"hydroxychloroquine', '"im', '"in', '"its', '"negative"', '"plandemia"', '"reclassified"', '"sarscov2', '"selfdiagnostic', '"this', '"vaccine"', '"well', '"weve', '"yes', '#', '##3600mg', '#1', '#2', '#2020presidentialelection', '#21dayschallenge', '#21dayslockdown', '#251', '#46', '#a', '#ableg', '#achristmascarol', '#acog2020', '#active', '#activecases', '#actnow', '#acuteflaccidmyelitis', '#afaracare', '#afoolat60', '#ahmedabad', '#airline', '#aliens', '#allergies', '#alllivesmatter', '#alzheimer’s', '#amazon', '#america', '#amitabhbachchan', '#andhrapradesh', '#anteater', '#antibiotics', '#aoc', '#apocalypse', '#appeal', '#area', '#arrestfauci', '#arrestgates', '#arunachalpradesh', '#arvindkejriwal', '#asco20', '#ashas', '#asians', '#asymptomatic', '#athens', '#auspol', '#autism', '#autoimmune', '#avifavir', '#aworldindisorder', '#azimpremji', '#backtoschool', '#bacteria', '#b

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Transform

In [84]:
bag_of_words = vectorizer_stop_words_stemmed.transform(X_train["tweet"])
bag_of_words = pd.DataFrame(bag_of_words.toarray(), columns = vectorizer_stop_words_stemmed.get_feature_names())

In [85]:
bag_of_words.shape

(5706, 19765)

# Árbol

In [86]:
from sklearn.tree import DecisionTreeClassifier

arbol = DecisionTreeClassifier(criterion = 'entropy', max_depth=4, random_state=42)
arbol = arbol.fit(bag_of_words, Y_train)

In [87]:
bag_of_words_Validation = vectorizer_stop_words_stemmed.transform(X_validation["tweet"])

In [88]:
from sklearn.metrics import accuracy_score
Y_pred = arbol.predict(bag_of_words_Validation)
print(accuracy_score(Y_pred,Y_validation))

0.7761037140854941


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Como se puede ver, aún luego de remover las stop words y hacer el lemmatizing, la accuracy es casi la misma, incluso un poco peor. Vamos a llevarlo a una red neuronal para ver si aprende mejor luego de este procesamiento